In [1]:
%load_ext autoreload
%autoreload 2

import unicodecsv as csv
import datetime
import feedparser as fp
import newspaper
import helpers
from newspaper import Source, Article
import multiprocessing 
import pandas as pd

In [2]:
%%time
# Compiling papers with using rss information, producing newspaper objects:
all_cnn_papers = helpers.compile_papers(rss_feeds_dict=helpers.CNN_LINKS_DICT)
all_bbc_papers = helpers.compile_papers(rss_feeds_dict=helpers.BBC_LINKS_DICT)
all_fox_papers = helpers.compile_papers(rss_feeds_dict=helpers.FOX_LINKS_DICT)
all_rt_papers = helpers.compile_papers(rss_feeds_dict=helpers.RT_LINKS_DICT)
# No msnbc rss

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_world.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_us.rss
[Parse lx

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://feeds.foxnews.com/foxnews/health
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://feeds.foxnews.com/foxnews/most-popular
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://feeds.foxnews.com/foxnews/section/lifestyle
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://feeds.foxnews.com/foxnews/most-popular
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://feeds.foxnews.co

ValueError: unsupported pickle protocol: 3

In [2]:
#all_rt_papers = helpers.compile_papers(rss_feeds_dict=helpers.RT_LINKS_DICT)
# paper = Source(rss_feed_url, memoize_articles=False)
# paper.build()
rss_feed_url = helpers.RT_LINKS_DICT['rss_feeds'][0]['feed_rss_link']
paper = Source(rss_feed_url, memoize_articles=False)

In [3]:
paper.build()

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] https://www.rt.com/rss/


ValueError: unsupported pickle protocol: 3

In [4]:
%%time
# Download articles, i.e. scrapeoff text and other data:
helpers.art_multithread_downloads(papers=all_cnn_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_bbc_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_fox_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_rt_papers, threads_per_source=2)

NameError: name 'all_rt_papers' is not defined

In [ ]:
master_paper_list = [all_cnn_papers, 
                     all_bbc_papers,
                     all_fox_papers,
                     all_rt_papers]

In [ ]:
%%time
# Parse all articles:
for paper_list in master_paper_list:
    for paper in paper_list:
        paper.parse_articles()

In [ ]:
def write_out_articles(newspaper_obj, filepath, writeout_type='conventional'):
    """
    TODO: finish this docstring
    :param newspaper_obj:
    :param filepath: str, filepath, WITHOUT the filename
    :param writeout_type: str, 'custom' or 'conventional'
    :return:
    """
    now = datetime.datetime.now()
    current_date = '{}_{}_{}'.format(str(now.year),
                                     str(now.month),
                                     str(now.day))
    if writeout_type == 'custom':
        csv_name = filepath + current_date + '_' + newspaper_obj.description \
                   + '.csv'
    else:
        csv_name = filepath + current_date + '_' + newspaper_obj.brand + '.csv'
    with open(csv_name, 'w') as outcsv:
        writer = csv.writer(outcsv, encoding='utf-8')
        if writeout_type == 'conventional':
            header = ['source_url', 'url', 'title', 'movies', 'text',
                      'keywords', 'meta_keywords', 'tags', 'authors',
                      'publish_date', 'summary', 'html', 'article_html',
                      'meta_description', 'meta_data', 'canonical_link']
        else:
            header = ['source_url', 'url', 'title', 'movies', 'text',
                             'keywords', 'meta_keywords', 'tags', 'authors',
                             'publish_date', 'summary', 'html', 'article_html',
                             'meta_description', 'meta_data', 'canonical_link',
                             'rss_title', 'rss_link', 'rss_id', 'rss_published',
                             'rss_published_parsed', 'rss_feedburner_origlink',
                             'paper_section_name']
        writer.writerow(header)
        for article in newspaper_obj.articles:
            source_url = article.source_url
            url = article.url
            title = article.title
            movies = article.movies
            text = article.text
            keywords = article.keywords
            meta_keywords = article.meta_keywords
            tags = article.tags
            authors = article.authors
            publish_date = article.published_date
            summary = article.summary
            html = article.html
            article_html = article.article_html
            meta_description = article.meta_description
            meta_data = article.meta_data
            canonical_link = article.canonical_link
            if writeout_type == 'custom':
                rss_title = article.additional_data['rss_title']
                rss_link = article.additional_data['rss_link']
                rss_id = article.additional_data['rss_id']
                rss_published = article.additional_data['rss_published']
                rss_published_parsed = \
                    article.additional_data['rss_published_parsed']
                rss_feedburner_origlink = \
                    article.additional_data['rss_feedburner_origlink']
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link, rss_title, rss_link,
                            rss_id, rss_published, rss_published_parsed,
                            rss_feedburner_origlink,
                            newspaper_obj.description]
            else:
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link]
            writer.writerow(data_row)

In [79]:
filepath = 'data/custom_data_pulls/'

In [80]:
%%time
for paper_list in master_paper_list:
    for paper in paper_list:
        write_out_articles(newspaper_obj=paper,
                                   filepath=filepath, 
                                   writeout_type='custom')

CPU times: user 10.7 s, sys: 4.07 s, total: 14.7 s
Wall time: 27 s


In [14]:
df = pd.read_csv('data/custom_data_pulls/2019_1_31_rt_main_page.csv')

In [16]:
df.text

0     Hundreds of chateaux, seemingly plucked out of...
1     European airplane maker Airbus has reassured t...
2     The US Department of Energy secretly shipped h...
3     Ottawa is considering slashing its embassy sta...
4     Neocon pundit Max Boot defended the unwinnable...
5     An 8-alarm fire broke out at a Marcal paper fa...
6     In an apparent bid to shield his case against ...
7     Thousands of homes and businesses have been le...
8     President Donald Trump’s bickering with US int...
9     A Kentucky jury has awarded Senator Rand Paul ...
10    Treacherous winter storm conditions caused a 2...
11    The Orthodox Church in America (OCA) has relea...
12    CNN’s Kamala Harris town hall broadcast, suppo...
13    Venezuelan President Nicolas Maduro called on ...
14    Washington has imposed sanctions on Albanisa, ...
15    A former UN envoy on Palestinian affairs has c...
16    A senior judge has resigned from the UN Intern...
17    Venezuela’s opposition leader and self-dec